# ML-for-Good-Hackathon
# Team Name: Vcare
# Participants: Sanjit Mehta, Naveena Chandwani, Rohith Rathod

### Import Common packages

In [1]:
import numpy as np
import pandas as pd
import re
import string
import math
import glob

### Import NLP related packages

In [2]:
#pip install contractions
import contractions
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from IPython.display import display

import multiprocessing

#pip install gensim
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict

from sklearn.cluster import KMeans

### Import Data and Drop duplicates

In [3]:
def read_data():
    path = 'C:/Users/NLP/data/' # use your path
    all_files = glob.glob(path + "/*.csv")

    df_list = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        df_list.append(df)

    df = pd.concat(df_list, axis=0, ignore_index=True)

    # Drop duplicates
    cols_req = ["focus_group_subtype", "focus_group_subtype_id", "doc_no_within_subtype", "question_id", 
                "question_text", "parent_num", "parent_answer"]

    df.drop_duplicates(inplace=True)
    df = df[cols_req]
    return df 

In [4]:
input_df = read_data()
input_df.head()

,focus_group_subtype,focus_group_subtype_id,doc_no_within_subtype,question_id,question_text,parent_num,parent_answer
0,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",5,Sure. Hi. My name is Parent 5. I have three ki...
1,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",1,"Hi everyone. My name is Parent 1, I have two b..."
2,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",3,Hi everybody. My name is Parent 3 and I have a...
3,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",4,"Hi, I'm Parent 4, I have a 15-year-old daughte..."
4,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",2,"Oh, I'm sorry. I lost connection, I couldn't h..."


# Preprocessing of text Data
1. Expand contraction
2. Case handling
3. Remove punctuations
4. Remove words and digits containing digits
5. Remove stop word
6. Lemmatization
7. Remove Extra Spaces 

#### 1. Expand contraction
Contraction is the shortened form of a word like don’t stands for do not, aren’t stands for are not. Like this, we need to expand this contraction in the text data for better analysis.

In [5]:
def expand_contraction(df,columns=[]):
    
    for col in columns:
        df[col] = df[col].apply(lambda text:contractions.fix(text))
        
    return df

#### 2. Case handling
If the text is in the same case, it is easy for a machine to interpret the words because the lower case and upper case are treated differently by the machine. for example, words like Ball and ball are treated differently by machine. So, we need to make the text in the same case and the most preferred case is a lower case to avoid such problems.

In [6]:
def case_handling(df,columns=[]):
    
    for col in columns:
        df[col] = df[col].str.lower() 
        
    return df       

#### 3. Remove punctuations
One of the other text processing techniques is removing punctuations. there are total 32 main punctuations that need to be taken care of. we can directly use the string module with a regular expression to replace any punctuation in text with an empty string

In [7]:
def remove_punctuations(df,columns=[]):
    
    for col in columns:
        df[col] = df[col].apply(lambda text: re.sub('[%s]' % re.escape(string.punctuation), '' , text))
        df[col] = df[col].apply(lambda text: text.replace("_"," "))
        
    return df   

#### 4. Remove words and digits containing digits
Sometimes it happens that words and digits combine are written in the text which creates a problem for machines to understand. hence, We need to remove the words and digits which are combined like game57 or game5ts7. This type of word is difficult to process so better to remove them or replace them with an empty string. we use regular expressions for this. 

In [8]:
def remove_words_dgits(df,columns=[]):
    
    for col in columns:
        df[col] = df[col].apply(lambda text: re.sub(" \d+",'',text))

    return df

#### 5. Remove stopword
Stopwords are the most commonly occurring words in a text which do not provide any valuable information. stopwords like they, there, this, where, etc are some of the stopwords. NLTK library is a common library that is used to remove stopwords and include approximately 180 stopwords which it removes. If we want to add any new word to a set of words then it is easy using the add method.

In [9]:
def remove_stopwords(df, columns=[]):
    
    stop_words = set(stopwords.words('english'))
    
    def remove_sw(text):
        txt_output = " ".join([word for word in str(text).split() if word not in stop_words])
        return txt_output
    
    for col in columns:
        df[col] = df[col].apply(lambda text: remove_sw(text))
    
    return df

#### 6. Lemmatization
Lemmatization is similar to stemming, used to stem the words into root word but differs in working. Actually, Lemmatization is a systematic way to reduce the words into their lemma by matching them with a language dictionary.

In [10]:
def lemmatize_words(df, columns=[]):
    
    lemmatizer = WordNetLemmatizer()
    
    def lemmatize(text):
        text_output = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
        return text_output
    
    for col in columns:
        df[col] = df[col].apply(lambda text: lemmatize(text))
        
    return df

#### 7. Remove Extra Spaces 
Most of the time text data contain extra spaces or while performing the above preprocessing techniques more than one space is left between the text so we need to control this problem. regular expression library performs well to solve this problem

In [11]:
def remove_extra_spaces(df,columns=[]):
    
    for col in columns:
        df[col] = df[col].apply(lambda text: re.sub(' +', ' ', text))
        
    return df 

In [12]:
def data_preprocessing(df, columns=[]):
    
    df = expand_contraction(df,columns)
    df = case_handling(df,columns) 
    df = remove_punctuations(df,columns)
    df = remove_words_dgits(df,columns)  
    df = remove_stopwords(df,columns) 
    df = lemmatize_words(df, columns)
    df = remove_extra_spaces(df,columns) 
    
    return df

## Data preprocessing function call

In [13]:
# Preprocess the data
columns=['parent_answer']
output_df =  data_preprocessing(input_df, columns)

## Unsupervised Sentiment Analysis

In [14]:
output_df['parent_answer'] =  output_df['parent_answer'].apply(lambda text: text.split())
sent = [row for row in output_df.parent_answer]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]

In [15]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     #size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

w2v_model.build_vocab(sentences, progress_per=50000)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
w2v_model.init_sims(replace=True)
w2v_model.save("word2vec.model")

<ipython-input-15-0a943fa732d4>:12: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [16]:
file_export = output_df.copy()
file_export['old_parent_answer'] = file_export.parent_answer
file_export.old_parent_answer = file_export.old_parent_answer.str.join(' ')
file_export.parent_answer = file_export.parent_answer.apply(lambda x: ' '.join(bigram[x]))
file_export[['parent_answer']].to_csv('cleaned_dataset.csv', index=False)

# KMeans clustering

In [17]:
word_vectors = Word2Vec.load("word2vec.model").wv
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))
#word_vectors.similar_by_vector(model.cluster_centers_[1], topn=30, restrict_vocab=None)

In [18]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [19]:
words = pd.DataFrame(word_vectors.index_to_key)
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [20]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

# Predictions

In [21]:
# Read the files
final_file = pd.read_csv('cleaned_dataset.csv')
sentiment_map = pd.read_csv('sentiment_dictionary.csv')
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))
file_weighting = final_file.copy()

### Getting tfidf scores of words in every sentence, and replacing them with their associated tfidf weights:

In [22]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
tfidf.fit(file_weighting.parent_answer)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(file_weighting.parent_answer)

C:\Apps\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


### Replacing words in sentences with their tfidf scores

In [23]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.parent_answer.split()))

### Replacing words in sentences with their sentiment score

In [24]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [25]:
# Replacing words in sentences with their tfidf scores
replaced_closeness_scores = file_weighting.parent_answer.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))
replaced_tfidf_scores = file_weighting.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)

# Merging both previous steps and getting the predictions:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.parent_answer]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'parent_answer']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')

## Classifying Parents response either Positive or Negative

In [26]:
replacement_df['sentiment'] = ['positive' if val == 1 else 'negative' for val in replacement_df['prediction']]
final_df = pd.merge(input_df, replacement_df['sentiment'], left_index=True, right_index=True)
final_df.head()

,focus_group_subtype,focus_group_subtype_id,doc_no_within_subtype,question_id,question_text,parent_num,parent_answer,sentiment
0,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",5,"[sure, hi, name, parent, three, kid, nine, boy...",negative
1,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",1,"[hi, everyone, name, parent, two, boy, ayearol...",positive
2,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",3,"[hi, everybody, name, parent, anyearold, daugh...",negative
3,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",4,"[hi, parent, ayearold, daughter, ayearold, son...",positive
4,gaming_group,1,1,1,"So, I was thinking we could start by just goin...",2,"[oh, sorry, lost, connection, could, hear, any...",positive


In [27]:
final_df.to_csv('focus_group_sentiment_output.csv', index=False)